In [4]:
import os
import random
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


In [5]:
# Directory where videos are stored
DATA_DIR = "data/sport"

# Mapping classes
classes = sorted(os.listdir(DATA_DIR))  # Assuming folder names are class names
print("Classes:", classes)

# Load videos and labels
data = []
labels = []

for category in classes:
    videos = os.listdir(os.path.join(DATA_DIR, category))
    for video in videos:
        video_path = os.path.join(DATA_DIR, category, video)
        # You might want to extract features or frames from videos here
        # For example, capturing one frame for simplicity
        cap = cv2.VideoCapture(video_path)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (224, 224))  # Resizing to a fixed size
            data.append(frame)
            labels.append(classes.index(category))
        cap.release()

data = np.array(data)
labels = np.array(labels)


Classes: ['barbell biceps curl', 'bench press', 'chest fly machine', 'deadlift', 'decline bench press', 'hammer curl', 'hip thrust', 'incline bench press', 'lat pulldown', 'lateral raise', 'leg extension', 'leg raises', 'plank', 'pull Up', 'push-up', 'romanian deadlift', 'russian twist', 'shoulder press', 'squat', 't bar row', 'tricep Pushdown', 'tricep dips']


In [6]:
# Normalize pixel values
data = data / 255.0

# Convert labels to one-hot encoding
labels = to_categorical(labels, num_classes=len(classes))

In [7]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


In [8]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(classes), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [9]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


Epoch 1/10
17/17 [==============================] - 7s 381ms/step - loss: 7.3292 - accuracy: 0.0710 - val_loss: 3.0720 - val_accuracy: 0.1069
Epoch 2/10
17/17 [==============================] - 6s 346ms/step - loss: 2.9843 - accuracy: 0.1171 - val_loss: 2.8445 - val_accuracy: 0.1145
Epoch 3/10
17/17 [==============================] - 6s 346ms/step - loss: 2.5241 - accuracy: 0.2610 - val_loss: 2.6470 - val_accuracy: 0.3206
Epoch 4/10
17/17 [==============================] - 6s 348ms/step - loss: 1.7040 - accuracy: 0.5547 - val_loss: 2.4498 - val_accuracy: 0.3130
Epoch 5/10
17/17 [==============================] - 6s 350ms/step - loss: 0.8073 - accuracy: 0.8042 - val_loss: 2.3125 - val_accuracy: 0.3817
Epoch 6/10
17/17 [==============================] - 6s 348ms/step - loss: 0.3003 - accuracy: 0.9386 - val_loss: 2.4346 - val_accuracy: 0.4275
Epoch 7/10
17/17 [==============================] - 6s 349ms/step - loss: 0.0865 - accuracy: 0.9923 - val_loss: 2.6449 - val_accuracy: 0.4351
Epoch 

In [10]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy*100}%")


5/5 [==============================] - 0s 66ms/step - loss: 2.8788 - accuracy: 0.4198
Test accuracy: 41.98473393917084%


In [11]:
model_save_path = "sport_model.h5"
model.save(model_save_path)

/home/chrysalis/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
